# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem

from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод
Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита
***
для удобства восприятия поменяла название столбцов
income_type >> type
total_income >> salary

***
нужно проверить тип данных в таблице. Внутри  в столбце образования проблемы с регистром, неудобно читать отрицательные значения в столбце

In [4]:
data = data.rename(columns={'income_type': 'type', 'total_income':'salary'}) 

In [5]:
data.tail(20) #проверить результат переименования и заодно посмотреть с конца

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,debt,salary,purpose
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21506,1,-1556.249906,33,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145541.993320,свадьба
21507,1,-79.832064,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180.279152,сделка с подержанным автомобилем
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
type                21525 non-null object
debt                21525 non-null int64
salary              19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
type                   0
debt                   0
salary              2174
purpose                0
dtype: int64

In [8]:
data[data['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,debt,salary,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [9]:
#проверим есть ли какая-то связь с типом
day_employed_salary = data[data['days_employed'].isna()].groupby('type').count() 

In [10]:
display(day_employed_salary)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,salary,purpose
type,,,,,,,,,,,
госслужащий,147,0,147,147,147,147,147,147,147,0,147
компаньон,508,0,508,508,508,508,508,508,508,0,508
пенсионер,413,0,413,413,413,413,413,413,413,0,413
предприниматель,1,0,1,1,1,1,1,1,1,0,1
сотрудник,1105,0,1105,1105,1105,1105,1105,1105,1105,0,1105


### Вывод
данные в строках в столбце "общий трудовой стаж" и "доход" отсутсвуют (Nan).  Им допустимо присвоить целочисленные значения и заменить медианным значением по каждому из типа, так как они все разные (для этой проверки вывела количество по типам). 

In [11]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,salary
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


###### вывод
- странные значения в столбце children - минусовое значение возможно тех.ошибка, 20 детей возможны, но выглядит маловероятным, поэтому нужно посмотреть количество
- аномальное среднее значение в days_employed, нужно проверить где ошибка
- возраст минимальный равен 0, предполагаю что ошибка или был не указан (хотя если кредит, возраст является обязательным полем, может быть тех ошибка или кто-то тестировал данные)
- отрицательные значения в days_employed непонятные и встречаются не во всем столбце,после перевода возможно прояснится понимание. 

#не получилось понять как вывести сразу по всем столбцам , поэтому смотрю отдельно

In [12]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [13]:
data[data['days_employed'] > 0].shape[0] #посчитаю положительные значения в столбце

3445

In [14]:
#не поняла как проверить как распределяются по типам положительные значения столбца days_employed, поэтому сделала группировкой и медианным значением посмотреть аномалии
data.groupby('type')['days_employed'].median()

type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

In [15]:
#посчитала еще для себя распределение дохода больше 0 как происходит по типам
data[data['days_employed'] > 0].groupby('type')['days_employed'].count()

type
безработный       2
пенсионер      3443
Name: days_employed, dtype: int64

In [16]:
data[data['days_employed'] < 0].shape[0] #количество отрицательных значений

15906

In [17]:
#проверим сколько людей с возрастом 0
data[data['dob_years'] == 0].shape[0]

101

In [18]:
#сделаю сводную методом agg
data_grouped_dob = data.groupby(['dob_years','type']).agg({'days_employed':'count'})


In [19]:
data_grouped_dob

days_employed
dob_years type                      
0         госслужащий              6
          компаньон               18
          пенсионер               17
          сотрудник               50
19        госслужащий              1
...                              ...
73        пенсионер                6
74        компаньон                1
          пенсионер                4
          сотрудник                1
75        госслужащий              1

[221 rows x 1 columns]

In [20]:
#сделаю категории по возрасту
def dob_years_group(dob_years):
    if dob_years <= 0:
        return 'сбой'
    if dob_years <= 18:
        return 'молодые'
    if dob_years <= 65:
        return 'взрослые'
    else:
        return 'пенсионеры' 
data['dob_years_group'] = data['dob_years'].apply(dob_years_group)

In [21]:
data['dob_years_group'] = data['dob_years'].apply(dob_years_group)

In [22]:
data['dob_years_group'].value_counts()

взрослые      20719
пенсионеры      705
сбой            101
Name: dob_years_group, dtype: int64

**Вывод**
- значения -1 и 20 в столбце children после просмотра кол-ва считаю технической ошибкой, которое нужно исправить (отрицательное значение - возможно лишний минус или остутствие ребенка (предположу что знак минуса лишний), в значении 20 - возможно лишний 0)
- отрицательных значений в days_employed больше половины, по логике так не может быть, скорее всего минус - это ошибка (учитывая что в программе обучения был метод для модуля). Положительные значения для безработных и пенсионеров,  плюс слишком большие (для безработных особенно), какое-то значение нарушает выборку.
- нулевой возраст - считаю тех ошибкой, как и раньше. Обычно это обязательное значение для кредита, соответственно данные не подгрузились, нужно будет заменить/вставить данные (для основного вопроса на первый взгляд этот столбец кажется неинформативным)

# Вывод итоговый
- нужно обработать Nan в days_employed и salary в соответсвии с типом (считаю медиану в зп, так как средний показатель может быть не совсем корректным)
- поменять тип данных после обработки days_employed с помощью int() на целочисленные
- значения в столбце days_employed в безработных и пенсионерах выглядят странными, нужно проверить корреляцию с другими столбцами ( скорее всего с возрастом) 
- исправить значения в столбце children и разбить по группам (для итогового вопроса понадобится)
- исправить регистр в столбце education 
- исправить нулевой возраст в dob_years и разбить на группы по возрастам, предположим что кредит до 18 лет не дадут (зависит от требований конкретного банка), вместо 0 должны быть значения из группы (молодые), которые не показала функция,  поставим 18


## Шаг 2. Предобработка данных

### Обработка пропусков

###### Заполняем пропуски NaN:
-'days_employed': NaN заменяем ;
-'salary': NaN заменяем на среднее значение по зарплате по группе 'type'.

In [23]:
#выясним пропуски до
data['days_employed'].isna().sum()

2174

In [24]:
#логично проверить в 1 части, но получилось тут. Проверяю аномалию
data[data['days_employed'].notnull()]['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 19351, dtype: float64

In [25]:
#смотрю подробно строку с ошибкой
anomal = data.loc[21521]

In [26]:
#фреb с группированными значениями
medians = (data.groupby(['type']).agg({'days_employed': 'median'}).rename(columns={'days_employed':'median_days_employed'}))

In [27]:
#пристегиваем к исходному фрейму и получится в итоге дополнительная колонка со средним значением
data = data.merge(medians, on = ['type'])

In [28]:
#заменяю Nan на медианы
data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed']

data['days_employed'] = data['days_employed'].fillna(data.groupby('type')['days_employed'].transform('median')) #пыталась попробовать метод, но как-то криво сработал

display(data[['type', 'days_employed', 'dob_years']].head(5))

In [29]:
data['days_employed'].isna().sum()

0

In [30]:
#пропуски в доходе до
data['salary'].isna().sum()

2174

In [31]:
#фреb с группированными значениями
medians2 = (data.groupby(['type']).agg({'salary': 'median'}).rename(columns={'salary':'median_salary'}))

In [32]:
#пристегиваем к исходному фрейму и получится в итоге дополнительная колонка со средним значением
data = data.merge(medians2, on = ['type'])

In [33]:
#заменяю Nan на медианы
data.loc[data['salary'].isna(), 'salary'] = data.loc[data['salary'].isna(), 'median_salary']

In [34]:
#пропуски в доходе после
data['salary'].isna().sum()

0

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 15 columns):
children                21525 non-null int64
days_employed           21525 non-null float64
dob_years               21525 non-null int64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
type                    21525 non-null object
debt                    21525 non-null int64
salary                  21525 non-null float64
purpose                 21525 non-null object
dob_years_group         21525 non-null object
median_days_employed    21525 non-null float64
median_salary           21525 non-null float64
dtypes: float64(4), int64(5), object(6)
memory usage: 2.6+ MB


In [36]:
# 'education' в low-индекс
data['education'] = data['education'].str.lower()

<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> 
<br>Избавились от большей части дубликатов в данных)
</div>

In [37]:
#поменяем -1 в детях на 1 и 20 - на 2
data['children'] = data['children'].replace(-1, 1)


In [38]:
data['children'] = data['children'].replace(20, 2)

In [39]:
#проверим что вышло
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [40]:
#замена возраста на 18 (для попадания в группу молодые). По факту пошла бы к разработчикам, так как в реальной жизни так делать нельзя.
data.loc[data['dob_years'] == 0, 'dob_years'] = 18

In [41]:
#сделаю категории по возрасту
def dob_years_group(dob_years):
    if dob_years <= 0:
        return 'сбой'
    if dob_years <= 18:
        return 'молодые'
    if dob_years <= 65:
        return 'взрослые'
    else:
        return 'пенсионеры' 
data['dob_years_group'] = data['dob_years'].apply(dob_years_group)

In [42]:
data['dob_years_group'] = data['dob_years'].apply(dob_years_group)

In [43]:
data['dob_years_group'].value_counts()

взрослые      20719
пенсионеры      705
молодые         101
Name: dob_years_group, dtype: int64

**Вывод по 2.1**
- заменила nan в days_employed  и salary на среднее значение
- заменила странные данные в возрасте (теперь три группы которые подходят под кредитные требования)
- заменила странные данные в детях

### Замена типа данных

###### что делаю:
Месячную зарплату salary и стаж в днях days_employed переводим в 'int' после перевода по модулю

In [44]:
data[['days_employed']] = data[['days_employed']].abs()

In [45]:
display(data.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,debt,salary,purpose,dob_years_group,median_days_employed,median_salary
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,взрослые,-1574.202821,142594.396847
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,взрослые,-1574.202821,142594.396847
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,взрослые,-1574.202821,142594.396847
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,взрослые,-1574.202821,142594.396847
4,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,взрослые,-1574.202821,142594.396847


In [46]:
data['days_employed'] = data['days_employed'].astype('int')
data['salary'] = data['salary'].astype('int')

In [47]:
#проверим тип в колонках
data.dtypes

children                  int64
days_employed             int64
dob_years                 int64
education                object
education_id              int64
family_status            object
family_status_id          int64
gender                   object
type                     object
debt                      int64
salary                    int64
purpose                  object
dob_years_group          object
median_days_employed    float64
median_salary           float64
dtype: object

**Вывод**:
Все данные переведены, теперь нет отрицательных значений и целые значения более читабельны (для анализа нам остатки не нужны)

### Обработка дубликатов

In [48]:
#посчитаем количество дубликатов
data.duplicated().sum()

71

In [49]:
data[data.duplicated()].sort_values(by=['salary', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,debt,salary,purpose,dob_years_group,median_days_employed,median_salary
11715,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,взрослые,365213.306266,118514.486412
12012,0,365213,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба,взрослые,365213.306266,118514.486412
12137,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,взрослые,365213.306266,118514.486412
12548,0,365213,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,взрослые,365213.306266,118514.486412
12569,0,365213,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,взрослые,365213.306266,118514.486412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18765,0,1547,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы,взрослые,-1547.382223,172357.950966
19085,0,1547,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью,взрослые,-1547.382223,172357.950966
19178,1,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости,взрослые,-1547.382223,172357.950966
19542,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба,взрослые,-1547.382223,172357.950966


In [50]:
data = data.drop_duplicates()

In [51]:
data.duplicated().sum()

0

**Вывод**: после просмотра дубликатов можно предположить что данные задвоились (в реале хорошо бы проверить по номеру телефона/эмэйлу),  как практика считаем утверждение верным и удаляем.

### Лемматизация

In [52]:
#логично проверить цель кредита и посмотреть уникальные значения и сделать списоки
purpose_unique = data['purpose'].unique()

In [53]:
print(purpose_unique)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'образование' 'на проведение свадьбы' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку своего автомобиля'
 'на покупку подержанного автомобиля' 'сыграть свадьбу' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'получение образования'
 'заняться образованием' 'автомобиль' 'свадьба'
 'получение дополнительного образования' 'покупка недвижимости'
 'операции с жильем' 'сделка с подержанным автомобилем'
 'покупка своего жилья' 'операции с недвижимостью' 'свой автомобиль'
 'профильное образование' 'высшее образование' 'сделка с автомобилем'
 'покупка жилой недвижимости' 'на покупку автомобиля'
 'покупка жилья для сдачи' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'ремонт жилью'
 'получение высшего образования' 'заняться высшим образованием']


In [54]:
m = Mystem()
def lemm_group(purpose): 
    for word in purpose.split():
        lemmas = m.lemmatize(purpose)
        if 'свадьба'  in lemmas:         
            return 'потреб'           
        if 'недвижимость' in purpose or 'жилье' in lemmas: 
            return 'недвижимость'
        if 'автомобиль'in lemmas:
            return 'автомобиль'
        return 'другое' 

In [55]:
data['lemm_group'] = data['purpose'].apply(lemm_group) 
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,debt,salary,purpose,dob_years_group,median_days_employed,median_salary,lemm_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,взрослые,-1574.202821,142594.396847,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослые,-1574.202821,142594.396847,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,взрослые,-1574.202821,142594.396847,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,взрослые,-1574.202821,142594.396847,другое
4,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,взрослые,-1574.202821,142594.396847,другое
5,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,взрослые,-1574.202821,142594.396847,потреб
6,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,взрослые,-1574.202821,142594.396847,недвижимость
7,0,792,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069,покупка коммерческой недвижимости,взрослые,-1574.202821,142594.396847,другое
8,0,1846,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458,приобретение автомобиля,взрослые,-1574.202821,142594.396847,автомобиль
9,1,972,26,среднее,1,женат / замужем,0,F,сотрудник,0,116820,строительство собственной недвижимости,взрослые,-1574.202821,142594.396847,другое


In [56]:
data['lemm_group'].unique()

array(['недвижимость', 'автомобиль', 'другое', 'потреб'], dtype=object)

**Вывод**
значения из столбца с целями кредита после лемматизации превратились в 4 категории. Для этого сначала лемматизировали данные из столбца как в теории, потом отдельная функция, которая проходит по ключам (п/с в одну вместить не получилось)

### Категоризация данных

In [57]:
#сделаем словарь для проверки

In [58]:
family_dict = data[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
print(family_dict.head()) 

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 3              в разводе
3                 4  Не женат / не замужем
4                 2         вдовец / вдова


In [59]:
data['salary'].median()

142594.0

In [60]:
data['salary'].min()

20667

**План**: <br/> Делаю категории для ответов на вопросы проекта.<br/> 
Группируем по доходу на три категории (исходя из средних значений)
-'низкий' (<= 55000р.);
-'средний' (55000 < x <= 150000);
-'высокий' (150000 < x);


Группируем по по возрасту:
- до 20 года
- 20-45 лет
- 45-65 лет
- более 65 лет

по количеству детей :
- 0 детей
- 1-2 ребенка
- 3 и больше

In [61]:
def salary_group(row):
    if row['salary'] <= 55000:
        return 'низкий'
    if 55000 < row['salary'] <= 150000:
        return 'средний'
    return 'высокий'



def dob_years_group(row):
    if row['dob_years'] < 20:
        return 'до 20 лет'
    if 20 <= row['dob_years'] < 45:
        return '20-45 лет'
    if 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    return 'более 65 лет'

    
def children_group(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
    
def children_group_2(row):
    if row['children'] > 0:
        return 'есть дети'
    else:
        return 'нет детей'

In [62]:
data['salary_group'] = data.apply(salary_group, axis=1)
data['dob_years_group'] = data.apply(dob_years_group, axis=1)
data['children_group'] = data.apply(children_group, axis=1)
data['children_group_2'] = data.apply(children_group_2, axis=1)

In [63]:
print(data[['salary_group', 'dob_years_group','children_group' ]].head(10))

  salary_group dob_years_group children_group
0      высокий       20-45 лет    1-2 ребенка
1      средний       20-45 лет    1-2 ребенка
2      средний       20-45 лет      нет детей
3      высокий       20-45 лет    многодетные
4      средний       45-65 лет      нет детей
5      средний       20-45 лет    1-2 ребенка
6      средний       20-45 лет      нет детей
7      средний       20-45 лет      нет детей
8      средний       45-65 лет      нет детей
9      средний       20-45 лет    1-2 ребенка


In [64]:
data['children_group_2' ]

0        есть дети
1        есть дети
2        нет детей
3        есть дети
4        нет детей
           ...    
21520    нет детей
21521    нет детей
21522    нет детей
21523    нет детей
21524    есть дети
Name: children_group_2, Length: 21454, dtype: object

**Вывод**:
данные по категориям разбиты

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [65]:
pivot1 =  data.pivot_table(index=['debt'], columns='children_group', values = 'family_status_id', aggfunc='count', margins=False)

In [66]:
print(pivot1)

children_group  1-2 ребенка  многодетные  нет детей
debt                                               
0                      6336          349      13028
1                       647           31       1063


In [67]:
children_0 = pivot1['нет детей'][1] / pivot1['нет детей'][0]
children_1 = pivot1['многодетные'][1] / pivot1['многодетные'][0]
children_more = pivot1['1-2 ребенка'][1] / pivot1['1-2 ребенка'][0]
print("{0:.2f}% Нет детей, есть просрочка".format(children_0*100))
print("{0:.2f}% 1-2 ребенка, есть просрочка".format(children_1*100))
print("{0:.2f}% многодетные, есть просрочка".format(children_more*100))

8.16% Нет детей, есть просрочка
8.88% 1-2 ребенка, есть просрочка
10.21% многодетные, есть просрочка


In [68]:
pivot1_2 =  data.pivot_table(index=['debt'], columns='children_group_2', values = 'family_status_id', aggfunc='count', margins=False)

In [69]:
print(pivot1_2)

children_group_2  есть дети  нет детей
debt                                  
0                      6685      13028
1                       678       1063


In [70]:
children_0_2 = pivot1_2['нет детей'][1] / pivot1_2['нет детей'][0]
children_1_2 = pivot1_2['есть дети'][1] / pivot1_2['есть дети'][0]

print("{0:.2f}% Нет детей, есть просрочка".format(children_0_2*100))
print("{0:.2f}% Есть дети, есть просрочка".format(children_1_2*100))

8.16% Нет детей, есть просрочка
10.14% Есть дети, есть просрочка


**Вывод**<br/> 
Наличие ребенка влияет на возможность просрочки по кредиту, увеличивая % 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [71]:
pivot2 = data.pivot_table(index=['debt'], columns='family_status', values='family_status_id', aggfunc='count')

In [72]:
pivot2

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt,,,,,
0,2536,1110,896,3763,11408
1,274,85,63,388,931


In [73]:
family = pivot2['женат / замужем'][1] / pivot2['женат / замужем'][0]
no_family = pivot2['Не женат / не замужем'][1] / pivot2['Не женат / не замужем'][0]
divorce = pivot2['в разводе'][1] / pivot2['в разводе'][0]
widow = pivot2['вдовец / вдова'][1] / pivot2['вдовец / вдова'][0]
partner = pivot2['гражданский брак'][1] / pivot2['гражданский брак'][0]

In [74]:
print("{0:.2f}% женат / замужем".format(family*100))
print("{0:.2f}% Не женат / не замужем".format(no_family*100))
print("{0:.2f}% в разводе".format(divorce*100))
print("{0:.2f}% вдовец / вдова".format(widow*100))
print("{0:.2f}% гражданский брак".format(partner*100))

8.16% женат / замужем
10.80% Не женат / не замужем
7.66% в разводе
7.03% вдовец / вдова
10.31% гражданский брак


**Вывод**<br/>
Люди со статусом Не женат / не замужем / гражданский брак чаще допускают просрочки

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [75]:
data.groupby('salary')['debt'].mean().to_frame().sort_values(by='debt')

,debt
salary,
145891,0.0
178101,0.0
178104,0.0
178120,0.0
178129,0.0
...,...
214263,1.0
214448,1.0
214462,1.0


In [76]:
pivot3 = data.pivot_table(index=['debt'], columns='salary_group', values='dob_years_group', aggfunc='count').reset_index()

In [77]:
pivot3

salary_group,debt,высокий,низкий,средний
0,0,9068,545,10100
1,1,763,35,943


In [78]:
average = pivot3['средний'][1] / pivot3['средний'][0]
short = pivot3['низкий'][1] / pivot3['низкий'][0]
high = pivot3['высокий'][1] / pivot3['высокий'][0]

In [79]:
print("{0:.2f}% cредний".format(average*100))
print("{0:.2f}% низкий".format(short*100))
print("{0:.2f}% высокий".format(high*100))

9.34% cредний
6.42% низкий
8.41% высокий


**Вывод**<br/>
заемщики с низким доходом оказывается более ответственно относятся к кредиту

- Как разные цели кредита влияют на его возврат в срок?

In [80]:
pivot4 = data.pivot_table(index=['debt'], columns='lemm_group', values='family_status_id', aggfunc='count')

In [81]:
pivot4

lemm_group,автомобиль,другое,недвижимость,потреб
debt,,,,
0,3903,7134,6538,2138
1,403,645,507,186


In [82]:
consumer = pivot4['потреб'][1] / pivot4['потреб'][0]
estate = pivot4['недвижимость'][1] / pivot4['недвижимость'][0]
another = pivot4['другое'][1] / pivot4['другое'][0]
auto = pivot4['автомобиль'][1] / pivot4['автомобиль'][0]
print("{0:.2f}% потреб".format(consumer*100))
print("{0:.2f}% недвижимость".format(estate*100))
print("{0:.2f}% другое".format(another*100))
print("{0:.2f}% автомобиль".format(auto*100))

8.70% потреб
7.75% недвижимость
9.04% другое
10.33% автомобиль


**Вывод**
ипотечный кредит для банка несет меньше риска, чем остальные виды, что нормально для залогового кредита

## Шаг 4. Общий вывод

По факту выполнения была проанализирована база, с самого начала основными методами просмотрена основная информация,  были обнаружены основные аномалии в данных,  заполнены пропущенные значения, удалены дубликаты, выделены категории учитывая средние, мин и максимальные значения по столбцам для удобства восприятия (да и далее по категориям намного удобнее работать). С помощью сводных таблиц получилось посмотреть какие критерии влияют на возврат кредита, но разница в процентах конечно незначительная для реальных цифр.

так как это учебный проект - не стала удалять свои "попытки" и просмотры, так как делать по шагам пока для меня единственный вариант для понимания